Install required packages

In [ ]:
!pip install OpenAI

In [ ]:
!pip install pandas

In [ ]:
!pip install IPython

Import modules

In [ ]:
import json, os
from openai import OpenAI
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from IPython.display import display, Markdown
from openai.types.chat import ChatCompletionMessageParam

Mount Google drive to access JSON file and CSV file

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Load API Key and initilize OpenAI client

In [ ]:
file_name = '/content/drive/MyDrive/MidTermFiles/config.json'

with open(file_name, 'r') as file:
  config = json.load(file)
  os.environ['OPENAI_API_KEY'] = config.get("API_KEY")
  os.environ["OPENAI_BASE_URL"] = config.get("OPENAI_API_BASE")

In [ ]:

#@title LLM Function
def llm(system_prompt, user_prompt, model_name="gpt-4o-mini", temperature=0.0):

    try:

        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        prompt: list[ChatCompletionMessageParam] = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]

        response = client.chat.completions.create(
            model=model_name,
            messages=prompt,
            temperature=temperature
        )

        return response.choices[0].message.content

    except Exception as e:
        error_message = f"Sorry, I encountered the following error: {e}"
        print(error_message)
        return error_message

In [ ]:
#@title Load Alex's Emails CSV File

df = pd.read_csv("/content/drive/MyDrive/MidTermFiles/Alex_emails_march_04.csv", index_col="email_id", encoding='latin-1')

yesterday_date = pd.to_datetime("3/3/2025").strftime('%m/%d/%Y')

df['date_received'] = pd.to_datetime(df['date_received']).dt.strftime('%m/%d/%Y')

yesterday_emails = df[df['date_received'] == yesterday_date].reset_index(drop=True)
print(f"Filtered Email Count: {len(yesterday_emails)}")

df.shape

yesterday_emails.info()

In [ ]:
#@title Create Categories List

categories = ['Urgent & High_Priority Emails',
              'Deadline-Driven Emails',
              'Routine Updates & Check-ins',
              'Non-Urgent Informational Emails',
              'Personal & Social Emails',
              'Spam/Unimportant Emails'
              ]

In [ ]:
# @title System Prompt

system_prompt = """

You are an expert AI assistant tasked with accurately categorizing emails into one of the following six categories:

'Urgent & High_Priority Emails': Requires immediate attention, action, or response. Often contains keywords like "urgent," "important," "action required," or has a demanding tone.
'Deadline-Driven Emails': Pertains to a specific deadline, submission, or time-sensitive task. Look for dates, times, or phrases like "due by," "deadline," or "submit by."
'Routine Updates & Check-ins': Standard operational messages, status reports, meeting minutes, or simple check-ins that do not require an immediate or urgent response.
'Non-Urgent Informational Emails': General announcements, newsletters, articles, or FYI (For Your Information) content that is good to know but not actionable.
'Personal & Social Emails': Non-work-related messages from friends, family, or social event invitations.
'Spam/Unimportant Emails': Unsolicited junk mail, advertisements, or promotional content.

Your response MUST be only the category name from the list above. Do not add any other text, explanations, or punctuation.

Here are a few examples:

---
Email:
Subject: [URGENT] Dashboard Syncing Issues – Production Metrics Missing
Body: We’ve got a big issue right now—live production metrics aren’t syncing properly on the Orion Analytics Dashboard. Some numbers are missing, others just look wrong. We’re seeing data gaps in yesterday’s batch reports, and my team can’t verify production outputs.

Couple of things:

This started sometime overnight. No one touched the system, but by 7 AM today, we noticed discrepancies.
Not clear if this is an API issue or data processing lag. Can your team confirm?
Production is running blind right now—we need this resolved ASAP or at least tell us how to get accurate numbers another way.
Need an update in the next 24 hours, or I’ll have to escalate further. If you need someone from our IT side, let me know.
Category: Urgent & High_Priority Emails
---
---
Email:
Subject: Approval Request: Budget Approval Needed by EOD
Body: I hope you're doing well. As we approach the deadline for finalizing the budget, I wanted to bring this to your attention. We need approval by the end of today to ensure smooth execution of next quarter's projects.

Attached, you'll find the latest budget breakdown. Let me know if you have any concerns or if you'd like to discuss any of the allocations before we move forward. Your prompt approval will help us avoid unnecessary delays.

Looking forward to your confirmation
Category: Deadline-Driven Emails
---
---
Email:
Subject: Daily Update – Project Titan (March 3)
Body: Completed Today:
Finished API integration for the reporting module. Data sync tests are successful with minor latency (~200ms).
Resolved the UI responsiveness issue on smaller screens (tested across devices).
Security team reviewed latest patches, no critical vulnerabilities found.
In Progress / Challenges:
Load Testing: Found performance dips when concurrent users exceed 5,000. Investigating if it's DB bottleneck or caching issue.
Client Feedback Implementation: We’re waiting on John (Acme Manufacturing) to clarify requirements before finalizing UI changes.
Bug #3421 (Session Timeouts): Still reproducing intermittently—Tom’s team is debugging with logs.
Next Steps (Tomorrow):
Start final API optimization based on today’s load testing results.
Deploy hotfix for session timeouts (if root cause is confirmed).
Prep for Friday’s internal review with the leadership team.
Let me know if you need any specific details or adjustments.
Category: Routine Updates & Check-ins
---
---
Email:
Subject: 10X Your Dev Team's Productivity with AI – Free Trial Ends Soon!
Body: Are your developers spending too much time debugging and fixing code instead of innovating? Our AI-powered coding assistant can write, optimize, and debug your code in seconds!
Limited-time offer: Get 3 months FREE if you sign up today!
Claim Your Free Trial Now
Category: Spam/Unimportant Emails
---
"""

In [ ]:
#@title Loop through and categorize emails

if 'category' not in yesterday_emails.columns:
    yesterday_emails['category'] = None

email_data = yesterday_emails.to_dict('records')

results = []

for row in tqdm(email_data, total=len(email_data), desc='Processing emails'):

    user_prompt = f"""
    Please categorize the following email:

    Sender: {row['sender']}
    Subject: {row['subject']}
    Date Received: {row['date_received']}
    Recipient: {row['main_recipient']}
    Email Body: {row['body']}
    """

    category_by_llm = llm(system_prompt, user_prompt)

    row['category'] = category_by_llm if category_by_llm in categories else "Categorization_Failed"
    results.append(row)

categorized_df = pd.DataFrame(results)
pd.set_option('display.max_rows', None)
display(categorized_df)

# **Task 1a - Executive Dashboard**

In [ ]:
# @title System Prompt

system_prompt = """

You are an expert AI assistant tasked with creating a high-level executive summary from a provided list of emails. Your response must be structured, clear, and follow the specified format exactly.

You must organize the output into the following sections:

1.  **Executive Summary of Emails**: This is the main title.
2.  **Total Number of Emails Received**: Provide the total count of all emails.
3.  **Total Number of Emails from Yesterday**: State the total count of emails from the previous day. Note that all emails in this batch are from March 3, 2025.
4.  **Email Breakdown by Categories (Count Only)**: List each email category and its corresponding count. The category 'Routine Updates & Check-ins' should be rephrased as 'Routine Updates (Review & Acknowledge)'.
5.  **AI Conclusion**: This section should have two distinct sub-sections:
    * **Critical Emails Requiring Immediate Attention**: This includes the sum of 'Urgent & High_Priority Emails' and 'Deadline-Driven Emails'. List the counts for each of these categories and then provide a total.
    * **Emails That Can Be Reviewed Later**: This includes the sum of 'Routine Updates & Check-ins', 'Non-Urgent Informational Emails', 'Personal & Social Emails', and 'Spam/Unimportant Emails'. List the counts for each of these categories and then provide a total.
6.  **Summary Insights**: Write a brief paragraph that highlights the number of critical emails that require immediate attention versus those that can be reviewed later. Emphasize the workload and suggest prioritizing the urgent and deadline-driven items to mitigate risks.

Analyze the DataFrame provided in the user prompt to get the counts for each category.
"""

In [ ]:
# @title User Prompt

user_prompt = f"""

Please generate the executive summary based on the following email data.

Below is the attached DataFrame for analysis:


Below is the attached DataFrame for analysis:

{categorized_df.to_string()}
"""

In [ ]:
# @title  Calling the model and displaying the summary

response_1 = llm(system_prompt, user_prompt)        # llm is the model using gpt-4o-mini
response_1

from IPython.display import display, Markdown
display(Markdown(response_1))

## **Task 1b - Urgent Emails from Yesterday (Must do first today)**

In [ ]:
# @title System Prompt

system_prompt = """
You are an expert AI assistant tasked with summarizing urgent and high_priority emails.
Your goal is to extract key information from each email and present it in a clear, concise, and actionable format.

For each email, you must provide the following details:
- **Subject**: The subject line of the email.
- **Received**: The date the email was received.
- **Sender Name**: The name of the sender.
- **Summary**: A brief summary of the email's content and its main purpose.
- **Next Step**: A specific, actionable step that needs to be taken in response to this particular email.

Ensure that the output is formatted as a numbered list, with each email summary clearly delineated.
Do not include any introductory or concluding remarks outside of the formatted summaries.
"""

In [ ]:
#@title Filter out the emails that are urgent and high-priority

urgent_emails = categorized_df[categorized_df['category'] == 'Urgent & High_Priority Emails']

urgent_df = pd.DataFrame(urgent_emails)

In [ ]:
# @title User Prompt
user_prompt = f"""
Below is the attached DataFrame, which contains all the emails that needs to be summarized:

{urgent_df.to_string()}
"""

In [ ]:
# @title  Calling the model and display the summary

response_2 = llm(system_prompt, user_prompt)

from IPython.display import display, Markdown
display(Markdown(response_2))

## **Task 1c - Deadline-Driven Emails from Yesterday (Needs Attention Today)**

In [ ]:
# @title System Prompt

system_prompt = """
You are an expert AI assistant tasked with summarizing deadline-driven emails from yesterday.
Your goal is to extract key information from each email and present it in a clear, concise, and actionable format.

For each email, you must provide the following details:
- **Subject**: The subject line of the email.
- **Received**: The time and date the email was received.
- **Sender Name**: The name of the person who sent the email.
- **Summary**: A concise summary of the email's content.
- **Next Step**: A specific, actionable step that needs to be taken based on the email, with a clear focus on meeting deadlines. Each 'Next Step' must explicitly mention a specific deadline for today or require action to meet an impending delivery timeline.

Ensure that the output is formatted as a numbered list, with each email summary clearly delineated, as follows:

1.  Email Exceeding Deadline / Email Requiring Action Today
    * **Subject**: (the emails subject line)
    * **Received**: (the time and date the email was received)
    * **Sender Name**: (the name of the person who sent the email)
    * **Summary**: (a concise summary of the emails content)
    * **Next Step**: (a specific action that needs to be taken based on the email, with a clear focus on meeting deadlines)

After listing all the email summaries, you must provide a final count and a summary of actionable items in the following format:

**Final Count of Deadline-Driven Emails**
* Total Deadline-Driven Emails: (Count, including a note if any have exceeded their deadline)

**Summary of Actionable Items**
* (List all the "Next Step" actions from the summarized emails as bullet points, combining similar actions if appropriate.)

Do not include any introductory or concluding remarks outside of the formatted summaries and the final count/actionable items section.
"""

In [ ]:
# @title Filtering out the emails that are Time Sensitive & Deadline-Driven
deadline_emails = categorized_df[categorized_df['category'] == 'Deadline-Driven Emails']

deadline_df = pd.DataFrame(deadline_emails)

In [ ]:
# @title User Prompt


# Write your user prompt here
user_prompt = f"""

Below is the attached DataFrame, which contains all the emails that needs to be summarized:

{deadline_df.to_string()}

"""

In [ ]:
# @title  Calling the model and displaying the summary

response_3 = llm(system_prompt, user_prompt)

from IPython.display import display, Markdown
display(Markdown(response_3))


## **Task 2 - AI Generated "First Response" drafts for critical email**

In [ ]:
# @title System Prompt

system_prompt = """
You are an expert AI assistant tasked with drafting professional and contextually relevant first responses to critical emails.
Your drafts must acknowledge the sender's request, address key points or queries from the original email, and provide a clear next step or decision.
Maintain a polite, formal, and professional tone, adhering to corporate communication standards.

For each email, generate a draft in the following format:

Subject: (subject of the email)
Sender Name: (name of the sender)
AI Drafted Reply: (the AI generated response)

Do not include any introductory or concluding remarks outside of this specified format.
"""

In [ ]:
# @title Filtering out the emails that are Critical Emails, i.e. ('Urgent & High-Priority Emails' + 'Deadline-Driven Emails')

critical_emails = categorized_df[categorized_df['category'].isin(['Urgent & High-Priority Emails', 'Deadline-Driven Emails'])]

critical_df = pd.DataFrame(critical_emails)

In [ ]:
# @title User Prompt

# You may format the classifying method according to you


# Write your user prompt here
user_prompt = f"""
Below is a DataFrame containing critical emails for which I need AI-drafted first responses. Please generate a professional draft for each email, following the specified format.

{critical_df.to_string()}
"""

In [ ]:
# @title  Calling the model and display the summary

response_4 = llm(system_prompt, user_prompt)
from IPython.display import display, Markdown
display(Markdown(response_4))

## **Task 3 - Evaluation**

In [ ]:
# @title Evaluation System Prompt

eval_system_prompt = """
You are an expert AI Quality Assurance Analyst. Your task is to critically evaluate an AI-generated email draft based on the original email it is responding to. You must assess the draft based on the six key dimensions provided below.

Your evaluation must be returned STRICTLY in the specified JSON format. Do not include any introductory text, markdown formatting, or any content outside of the JSON object.

**Evaluation Criteria & Scoring:**

1.  **Relevance (Score 1-5):** How well does the draft address the core request, questions, and urgency of the original email?
    * 1: Poor - Completely misses the point of the original email.
    * 3: Average - Addresses the main point but misses key details or context.
    * 5: Excellent - Perfectly understands and addresses all key aspects of the original email.

2.  **Clarity (Score 1-5):** How clear, concise, and understandable is the language in the draft? Is it free of jargon and ambiguity?
    * 1: Poor - Very confusing and poorly worded.
    * 3: Average - Generally understandable but could be clearer or more concise.
    * 5: Excellent - Exceptionally clear, professional, and easy to understand.

3.  **Actionability (Score 1-5):** Does the draft clearly state the next step, decision, or a commitment to action?
    * 1: Poor - Vague, non-committal, and provides no clear next step.
    * 3: Average - Hints at a next step but lacks specific commitment or timeline.
    * 5: Excellent - Provides a clear, specific, and actionable next step.

4.  **Strengths:** In one sentence, describe the single biggest strength of the generated draft.

5.  **Improvements:** In one sentence, suggest the most important area for improvement.

6.  **Overall_Justification:** Provide a 2-3 line summary of your evaluation, justifying the scores given.

**Required JSON Output Format:**

{
  "Relevance": {"score": 0, "justification": ""},
  "Clarity": {"score": 0, "justification": ""},
  "Actionability": {"score": 0, "justification": ""},
  "Strengths": "",
  "Improvements": "",
  "Overall_Justification": ""
}
"""

In [ ]:
# @title Evaluation Function & User Prompt

def evaluate_summary(eval_system_prompt, eval_user_prompt, eval_model="gpt-4o-mini", temperature=0.0):
    try:

        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        modified_prompt: list[ChatCompletionMessageParam] = [
            {'role': 'system', 'content': eval_system_prompt},
            {'role': 'user', 'content': eval_user_prompt}
            ]

        eval_response = client.chat.completions.create(
            model=eval_model,
            messages=modified_prompt,
            temperature=temperature
        )

        return eval_response.choices[0].message.content

    except Exception as e:
        error_message = f"Error evaluating prompt: {e}"
        print(error_message)
        return "{}"  # Return empty JSON structure on error

In [ ]:
# @title Evaluation for Each Generated Response (Updated)

# The response from Task 2 is a single string with drafts separated by "---"
# We split the string and remove any empty entries that result from the split.
evaluation_results = []

            # Dynamically create the user prompt with the original email and the generated draft
            eval_user_prompt = f"""
            Please evaluate the "AI-Generated Draft" based on its response to the "Original Email" using the specified JSON format.

            **Original Email:**
            - **Sender:** {original_email['sender']}
            - **Subject:** {original_email['subject']}
            - **Body:** {original_email['body']}

            ---

            **AI-Generated Draft to Evaluate:**
            {draft_to_evaluate}
            """

            # Call the evaluation function and append the JSON string to our results list
            evaluation_json_str = evaluate_summary(eval_system_prompt, eval_user_prompt)
            evaluation_results.append(evaluation_json_str)

        except Exception as e:
            print(f"An error occurred during evaluation for email at index {original_email_index}: {e}")
            # Append an empty JSON object on error to avoid breaking the script
            evaluation_results.append('{}')

print("\\nEvaluation complete.")

In [ ]:
# @title Converting JSON Scores into DataFrame

scores = []
justifications = []
strengths = []
improvements = []

for result in evaluation_results:
    try:
        # Load the JSON string into a Python dictionary
        result_dict = json.loads(result)

        # Extract scores, handling potential missing keys gracefully
        score_dict = {
            "Relevance": result_dict.get("Relevance", {}).get("score", None),
            "Clarity": result_dict.get("Clarity", {}).get("score", None),
            "Actionability": result_dict.get("Actionability", {}).get("score", None)
        }
        justification = result_dict.get("Overall_Justification", "NA")
        strength = result_dict.get("Strengths", "NA")
        improvement = result_dict.get("Improvements", "NA")

        # Append the extracted data to our lists
        scores.append(score_dict)
        justifications.append(justification)
        strengths.append(strength)
        improvements.append(improvement)

    except (json.JSONDecodeError, AttributeError) as e:
        print(f"Error parsing a result: {e}\\nResult string: {result}")
        # Append placeholder data if a result is malformed
        scores.append({"Relevance": None, "Clarity": None, "Actionability": None})
        justifications.append("Error parsing result")
        strengths.append("Error parsing result")
        improvements.append("Error parsing result")

# Create the final DataFrame
pd.set_option('display.max_colwidth', None)
df_scores = pd.DataFrame(scores)
df_scores["Strengths"] = strengths
df_scores["Improvements"] = improvements
df_scores["Justification"] = justifications

In [ ]:
# @title Display Final Evaluation Table
display(df_scores)